# Classificação 
# Descubrindo quem fez o ENEM 2016 apenas para treino

Alguns estudantes decidem realizar prova do ENEM de forma precoce, como um teste (coluna `IN_TREINEIRO`). Neste desafio, criaremos um modelo de classificação binária para tal feature, em que "0" é considerado como não realizou a prova de forma precoce e "1" é considerado como realizou de forma precoce. Para tal, recorreremos ao modelo Random Forest

### _Setup_

In [163]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [164]:
train = pd.read_csv('train.csv', index_col='Unnamed: 0')
test = pd.read_csv('test.csv')
ids = test.NU_INSCRICAO

### Visão Geral

In [165]:
print('Dataset de Treino com {0} observações e {1} colunas'.format(train.shape[0],train.shape[1]))
print('Dataset de Teste com {0} observações e {1} colunas'.format(test.shape[0],test.shape[1]))

Dataset de Treino com 13730 observações e 166 colunas
Dataset de Teste com 4570 observações e 43 colunas


In [166]:
train.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=166)

In [167]:
test.columns

Index(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC',
       'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA',
       'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
       'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047'],
      dtype='object')

In [168]:
train.isna().sum()[train.isna().sum()>0].sort_values(ascending=False)

NO_ENTIDADE_CERTIFICACAO       12092
CO_UF_ENTIDADE_CERTIFICACAO    12092
SG_UF_ENTIDADE_CERTIFICACAO    12092
Q041                           10792
SG_UF_ESC                       9448
TP_LOCALIZACAO_ESC              9448
TP_SIT_FUNC_ESC                 9448
CO_UF_ESC                       9448
NO_MUNICIPIO_ESC                9448
CO_MUNICIPIO_ESC                9448
CO_ESCOLA                       9448
TP_ENSINO                       9448
TP_DEPENDENCIA_ADM_ESC          9448
Q032                            7376
Q031                            7376
Q028                            7376
Q033                            7376
Q030                            7375
Q029                            7375
Q027                            7373
NU_NOTA_COMP1                   3597
NU_NOTA_LC                      3597
NU_NOTA_REDACAO                 3597
NU_NOTA_MT                      3597
TP_STATUS_REDACAO               3597
NU_NOTA_COMP5                   3597
TX_RESPOSTAS_LC                 3597
T

In [169]:
test.isna().sum()[test.isna().sum()>0].sort_values(ascending=False)

TP_DEPENDENCIA_ADM_ESC    3144
TP_ENSINO                 3144
Q027                      2437
NU_NOTA_REDACAO           1170
NU_NOTA_COMP5             1170
NU_NOTA_COMP4             1170
NU_NOTA_COMP3             1170
NU_NOTA_COMP2             1170
NU_NOTA_COMP1             1170
TP_STATUS_REDACAO         1170
NU_NOTA_LC                1170
NU_NOTA_CH                1112
NU_NOTA_CN                1112
dtype: int64

### Pré-processamento de Dados e Feature Engeenering 

Como extraíremos o resultado do dataset de teste, iremos filtrar o dataset de treino justamente com as features que o dataset de teste possui

In [170]:
features = test.columns.tolist()
features.append('IN_TREINEIRO')  # Manter a features 'IN_TREINEIRO' no Treino
train = train[features]
# Resever para o Alternativa - Feature Engeenering
train2 = train.copy()
test2 = test.copy()

Vamos conferir os dados faltantes (NAs) em termos absolutos e percentuais

In [173]:
train.isna().sum()[train.isna().sum()>0].sort_values(ascending=False)

TP_DEPENDENCIA_ADM_ESC    9448
TP_ENSINO                 9448
Q027                      7373
NU_NOTA_REDACAO           3597
NU_NOTA_COMP5             3597
NU_NOTA_COMP4             3597
NU_NOTA_COMP3             3597
NU_NOTA_COMP2             3597
NU_NOTA_COMP1             3597
TP_STATUS_REDACAO         3597
NU_NOTA_LC                3597
NU_NOTA_CH                3389
NU_NOTA_CN                3389
dtype: int64

In [174]:
test.isna().sum()[test.isna().sum()>0].sort_values(ascending=False)

TP_DEPENDENCIA_ADM_ESC    3144
TP_ENSINO                 3144
Q027                      2437
NU_NOTA_REDACAO           1170
NU_NOTA_COMP5             1170
NU_NOTA_COMP4             1170
NU_NOTA_COMP3             1170
NU_NOTA_COMP2             1170
NU_NOTA_COMP1             1170
TP_STATUS_REDACAO         1170
NU_NOTA_LC                1170
NU_NOTA_CH                1112
NU_NOTA_CN                1112
dtype: int64

In [175]:
train.isna().sum()[train.isna().sum()>0].sort_values(ascending=False)/train.shape[0]

TP_DEPENDENCIA_ADM_ESC    0.688128
TP_ENSINO                 0.688128
Q027                      0.536999
NU_NOTA_REDACAO           0.261981
NU_NOTA_COMP5             0.261981
NU_NOTA_COMP4             0.261981
NU_NOTA_COMP3             0.261981
NU_NOTA_COMP2             0.261981
NU_NOTA_COMP1             0.261981
TP_STATUS_REDACAO         0.261981
NU_NOTA_LC                0.261981
NU_NOTA_CH                0.246832
NU_NOTA_CN                0.246832
dtype: float64

In [176]:
test.isna().sum()[test.isna().sum()>0].sort_values(ascending=False)/test.shape[0]

TP_DEPENDENCIA_ADM_ESC    0.687965
TP_ENSINO                 0.687965
Q027                      0.533260
NU_NOTA_REDACAO           0.256018
NU_NOTA_COMP5             0.256018
NU_NOTA_COMP4             0.256018
NU_NOTA_COMP3             0.256018
NU_NOTA_COMP2             0.256018
NU_NOTA_COMP1             0.256018
TP_STATUS_REDACAO         0.256018
NU_NOTA_LC                0.256018
NU_NOTA_CH                0.243326
NU_NOTA_CN                0.243326
dtype: float64

Vamos retirar as variáveis que possuem mais de 50% de dados faltantes

In [177]:
train.drop(['TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO'], axis=1, inplace=True)
test.drop(['TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO'], axis=1, inplace=True)

Iremos descartar as variáveis referentes as dados do questionário socioeconômico, que são pouco relevantes para a análise.

In [178]:
train.drop(['NU_INSCRICAO','Q001', 'Q002', 'Q006', 'Q024','Q025', 'Q026', 'Q047'],axis=1, inplace=True)
test.drop(['NU_INSCRICAO','Q001', 'Q002', 'Q006', 'Q024','Q025', 'Q026', 'Q047'],axis=1, inplace=True)

Em seguida, dentre as variáveis com dados faltantas que ainda sobraram, nós deixamos apenas as notas das diferentes provas

In [179]:
lista = test.isna().sum().sort_values(ascending=False)[test2.isna().sum()>0].index.tolist()
remove = list(set(lista) - set(['NU_NOTA_LC','NU_NOTA_CH', 'NU_NOTA_CN',  'NU_NOTA_REDACAO']))
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

Em seguida, imputamos a nota 0 para aqueles estudantes que faltaram à prova (`TP_PRESENCA_XX` igual a 0) ou que foram eliminados (`TP_PRESENCA_XX` igual a 2). 

In [191]:
#Para o treino
train.NU_NOTA_CN.loc[train.TP_PRESENCA_CN == 0] = 0
train.NU_NOTA_CN.loc[train.TP_PRESENCA_CN == 2] = 0
train.NU_NOTA_CH.loc[train.TP_PRESENCA_CH == 0] = 0
train.NU_NOTA_CH.loc[train.TP_PRESENCA_CH == 2] = 0
train.NU_NOTA_LC.loc[train.TP_PRESENCA_LC == 0] = 0
train.NU_NOTA_LC.loc[train.TP_PRESENCA_LC == 2] = 0
train.NU_NOTA_REDACAO.loc[pd.isnull(train.NU_NOTA_REDACAO)] = 0

#Para o teste
test.NU_NOTA_CN.loc[test.TP_PRESENCA_CN == 0] = 0
test.NU_NOTA_CN.loc[test.TP_PRESENCA_CN == 2] = 0
test.NU_NOTA_CH.loc[test.TP_PRESENCA_CH == 0] = 0
test.NU_NOTA_CH.loc[test.TP_PRESENCA_CH == 2] = 0
test.NU_NOTA_LC.loc[test.TP_PRESENCA_LC == 0] = 0
test.NU_NOTA_LC.loc[test.TP_PRESENCA_LC == 2] = 0
test.NU_NOTA_REDACAO.loc[pd.isnull(test.NU_NOTA_REDACAO)] = 0


c:\users\dell\codenation\venv\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Quanto ao sexo dos participantes, apenas alteramos para o tipo da variável para que seja compatível com a leitura do algoritmo do modelo de classificação.

In [181]:
sexo= {'M':1,'F':2}
train['TP_SEXO'] = pd.Series(sexo[item] for item in train['TP_SEXO'])
test['TP_SEXO'] = pd.Series(sexo[item] for item in test['TP_SEXO'])

Por fim, realizamos o _One-hot Encoding_, que se trata de um técnica de conversão de variáveis categóricas para variáveis numérica, de maneira que seja compatível com o algoritmo do modelo. Nesse caso, recorremos à função nativa do Pandas `get_dummies` que altera o dataframe transformando cada categoria específica de cada variável categórica em uma _feature_ (uma coluna) no novo dataframe composto apenas dados binário, em que 0 corresponde à ausência da categoria e 1 à presença da categoria.

In [182]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# SMOTE

   Um problema frequente em modelos de classificação é a questão dos dados desbalanceados. O desbalanço dos dados no caso da classificação pode gerar um viés no modelo de modo que ele seja incapaz de captar a classe minoritária (que é, no caso, nosso interesse de análise), uma vez que o modelo aprende espertamente que um novo dado se refere à classe majoritária simplesmente pela sua maior quantidade. 
   Por exemplo, diante de um dataset composto por uma classe majoritária A, que representa em 96% dos dados, enquanto a classe minoritária B representa apenas 4%, o modelo irá classificar um novo dado como sendo A, 96% das vezes, o que consiste num desempenho ilusória.
   Há duas maneiras de convertar nessa situação: Undersampling, removendo dados da classe majoritária; Oversampling, reamostrando ados da classe minoritária.
   
   No nosso caso, lançacermos mão do oversampling, por meio do SMOTE. SMOTE se trata de uma técnica que adiciona dados sintéticos à classe minitoritária através da descoberta dos 𝑘 vizinhos mais próximos.


In [ ]:
# Distribuição dos valores dos estudantes que fizeram o ENEM como um teste
train.IN_TREINEIRO.value_counts()

In [ ]:
# Proporção
train.IN_TREINEIRO.value_counts()[1]/train.IN_TREINEIRO.value_counts()[0]

In [187]:
smote = SMOTE(sampling_strategy="minority")
X_smote, y_smote = smote.fit_resample(train.drop('IN_TREINEIRO', axis=1), train.IN_TREINEIRO)

#  Random Forest

A Random Forest consiste num algoritmo do tipo árvore de decisão, marcado pela ténica _bootstraping_ e pela subdivisão de features. A técnica de bootstraping consiste em reamostrar o conjunto de treinamento com reposição e treinar um modelo para cada uma das reamostragens. Após o treinamento dos vários modelos, a classe prevista pela maioria dos modelos é escolhida como classe final.

A subdivisão de features se trata de subdividir as features usadas durante o treinamento das árvores de decisão. Dessa forma, a cada split da árvore de decisão, apenas um subconjunto das features é considerado, o que constribui para a criação de  árvores mais variadas, e portanto, menos correlacionadas.Assim sendo, temos um erro total ainda menor.

In [188]:
randomforest = RandomForestClassifier()

In [189]:
randomforest.fit(X_smote, y_smote)

RandomForestClassifier()

In [192]:
predict = randomforest.predict(test)

In [ ]:
resposta = pd.DataFrame({'NU_INSCRICAO':ids, 'IN_TREINEIRO':predict})
resposta.to_csv('answer.csv', index=False)